# Perplexity

In [1]:
import numpy as np

def calculate_perplexity(probabilities):
    log_probs = np.log2(probabilities)
    avg_log_prob = np.mean(log_probs)
    perplexity = 2 ** (-avg_log_prob)
    return perplexity

In [2]:
true_sentence = "The quick brown fox jumps over the lazy dog"
sentence_1 = "The fast black cat jumps over the lazy dog"

s1_word_proba = [0.99, 0.85, 0.89, 0.99, 0.99, 0.99, 0.99, 0.99]
perplexity = calculate_perplexity(s1_word_proba)
print("Perplexity sentence 1:", perplexity)

Perplexity sentence 1: 1.04333190315947


In [3]:
sentence_2 = "The bold orange car drove by the lazy dog"

s2_word_proba = [0.99, 0.65, 0.13, 0.05, 0.21, 0.99, 0.99, 0.99]
perplexity = calculate_perplexity(s2_word_proba)
print("Perplexity sentence 2:", perplexity)

Perplexity sentence 2: 2.419227171949897


## Relationship to Cross Entropy

In [4]:
def cross_entropy(p, q):
    # Clip q to avoid log2(0) which is undefined
    q = np.clip(q, 1e-10, 1.0)
    H = -np.sum(p * np.log2(q))
    
    return H

n = len(s1_word_proba)

In [5]:
cross_entropy(np.ones(n), s1_word_proba)

0.48958543061604043

In [6]:
2**(cross_entropy(np.ones(n), s1_word_proba) / n )

1.04333190315947

In [7]:
calculate_perplexity(s1_word_proba)

1.04333190315947